In [11]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from torch.utils.data import DataLoader
cudnn.benchmark = True
plt.ion()   # interactive mode
from utils.SequenceDataset import SequenceDataset
from tqdm import tqdm
import torch.nn.functional as F
CLASSES=14
device = torch.device("cuda:0" if torch.cuda.is_available() else  "cpu")

In [2]:
import pickle
with open('./pickle/vids2label.p', 'rb') as fp:
    vids2label = pickle.load(fp)

In [3]:
train_set = SequenceDataset()
val_set = SequenceDataset(train=False)
train_set.__len__()

50

In [4]:
bs=1
dataset_sizes = {'train':train_set.__len__(),'val': val_set.__len__()}
train_loader = DataLoader(dataset=train_set,
                          batch_size=bs,
                          shuffle=True,
                          num_workers=4)
val_loader = DataLoader(dataset=val_set,
                          batch_size=bs,
                          shuffle=True,
                          num_workers=4)
dataloaders={'train':train_loader,'val':val_loader}

In [5]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
   
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            running_number_of_preds=0.0
            # Iterate over data.
            with tqdm(dataloaders[phase], unit="batch") as tepoch:
                batches=0
                for inputs, labels in tepoch:
                    tepoch.set_description(f"Epoch {epoch}")
                    inputs = inputs.to(device)
                    labels = labels.type(torch.LongTensor)
                    labels = labels.to(device)

                    # zero the parameter gradients
                    optimizer.zero_grad()
                    
                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        
                        outputs = model(inputs)
                        
                        outputs = torch.reshape(outputs, (-1,CLASSES))
                        
                        labels = labels.view(-1)
                        _, preds = torch.max(outputs, 1)
                       
                        loss = criterion(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                    batches+=1    
                    
                    tot_len=len(preds)
                    running_number_of_preds+=tot_len
                    #print(running_corrects)
                    tepoch.set_postfix(loss=running_loss/(running_number_of_preds), accuracy=100. * running_corrects.item()/(running_number_of_preds))

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / running_number_of_preds
            epoch_acc = running_corrects.double() / running_number_of_preds

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [6]:
class LSTM(nn.Module):
    def __init__(self):
        super(LSTM, self).__init__()        
        self.lstm = nn.LSTM(input_size=14, hidden_size=32, num_layers=2,bidirectional=True)
        #self.fc1 = nn.Linear(128, 32)
        self.fc2 = nn.Linear(64, CLASSES)
       
    def forward(self, x_3d):
        #x3d:  torch.Size([128, 32, 1, 80, 80])
        hidden = None
        toret  = []
        
        output, hidden = self.lstm(x_3d, hidden)
       

        x = F.relu(output)
        x = self.fc2(x)
        #print("x shape: ", x.shape)
            
            
        return x

In [7]:
model = LSTM()
model.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(model.parameters(), lr=0.01)

# Decay LR by a factor of 0.1 every 2 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=40, gamma=0.1)

In [8]:
model = train_model(model, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=80)

Epoch 0/99
----------


Epoch 0: 100%|██████████████████████████████████| 50/50 [00:00<00:00, 82.68batch/s, accuracy=34.7, loss=0.00126]


train Loss: 0.0013 Acc: 0.3475


Epoch 0: 100%|██████████████████████████████████| 20/20 [00:00<00:00, 61.66batch/s, accuracy=59.5, loss=0.00042]


val Loss: 0.0004 Acc: 0.5949

Epoch 1/99
----------


...

val Loss: 0.0002 Acc: 0.8071

Epoch 99/99
----------


Epoch 99: 100%|████████████████████████████████| 50/50 [00:00<00:00, 83.17batch/s, accuracy=83.9, loss=0.000403]


train Loss: 0.0004 Acc: 0.8386


Epoch 99: 100%|████████████████████████████████| 20/20 [00:00<00:00, 63.03batch/s, accuracy=84.3, loss=0.000187]

val Loss: 0.0002 Acc: 0.8431

Training complete in 1m 33s
Best val Acc: 0.868515


In [10]:
torch.save(model.state_dict(), './models/lstm.pt')